# Project: Wrangle OpenStreetMap data 
## By: Chaitanya Narayanavaram
In this project, I have used data wrangling techniques to audit, clean and analyze the data of an area from OpenStreetMap.


## Area: Houston, Texas
Data (.osm) available at: https://mapzen.com/data/metro-extracts/metro/houston_texas/

## Importing the necessary libraries:

In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint
import re

### Source files:

In [2]:
SAMPLE_FILE = "houston_texas_sample.osm" # shorter version of the original file (26.7 MB)
# This sample file is declared here, but the file is written below
OSM_FILE = "houston_texas.osm" # This is the original file (656 MB)

### Producing a smaller sample (houston_texas_sample.osm) of the original osm file (houston_texas.osm)

In [3]:
k = 15 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

### Analyzing the tags in the OSM file:
Below we can see what tags are present in the OSM file and how many times each tag occurs 

In [3]:
def analyze_tags(file):
    tag_types = defaultdict(int)
    for event, elem in ET.iterparse(file):
        tag_types[elem.tag] += 1
    return tag_types
analyze_tags(SAMPLE_FILE)

defaultdict(int,
            {'member': 1287,
             'nd': 250282,
             'node': 201894,
             'osm': 1,
             'relation': 164,
             'tag': 139297,
             'way': 24468})

### Tag patterns:

In [ ]:
lower = re.compile(r'^([a-z]|_)*$')
#  tags that contain only lowercase letters and are valid
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
# otherwise valid tags with a colon in their names
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
# tags with problematic characters


def key_type(element, keys):
    if element.tag == "tag":
        k_value = element.attrib['k']
        
        if lower.search(k_value):
            keys["lower"] += 1
        elif lower_colon.search(k_value):
            keys["lower_colon"] += 1
        elif problemchars.search(k_value):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
            # other tags that do not fall into the other three categories
        
    return keys


def process_map_tag_type(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map_tag_type(OSM_FILE)
print "Tags Patterns are: "
pprint.pprint(keys)

### Analyzing the keys in tags:
Below we can see what are the keys that occur in tags and how many times each of them occur.

In [14]:
def tag_keys(file):
    tag = defaultdict(int)
    for event, elem in ET.iterparse(file):
        if (elem.tag == 'way'):
            for e in elem.iter("tag"):
                    tag[e.attrib['k']] +=1
                
    return tag
tag_keys(OSM_FILE)
            

defaultdict(int,
            {'FIXME': 1064,
             'FIXME:hazmat': 2,
             'FIXME:oneway': 7,
             'NHS': 1425,
             'PASS_route': 1152,
             'RPM': 1,
             'TNRIS:OBJECTID': 1,
             'Texas_Trunk_System': 381,
             'UFID': 3,
             '_LINEARID_': 4,
             'abandoned': 9,
             'abandoned:highway': 41,
             'abandoned:junction': 1,
             'abandoned:leisure': 3,
             'abandoned:name': 3,
             'abandoned:ref': 1,
             'access': 12147,
             'access:bus': 74,
             'access:conditional': 6,
             'access:lanes:both_ways': 4,
             'addr:city': 4567,
             'addr:country': 289,
             'addr:full': 3,
             'addr:housename': 29,
             'addr:housenumber': 2935,
             'addr:inclusion': 20,
             'addr:interpolation': 20,
             'addr:postcode': 2315,
             'addr:state': 4220,
             'addr:

### Below is the function that will audit a particular field in the tags

In [8]:
def tag_audit(file, field):
    # This function will accept the file and the field that needs to be analyzed as the input
    # This function will return a dictionary with unique entries in the field (as keys) 
    # and the number of times this unique field has occurred (as values)
    tag = defaultdict(int)
    for event, elem in ET.iterparse(file):
        if (elem.tag == 'way'):
            for e in elem.iter("tag"):
                    if (e.attrib['k'] == field):
                        tag[e.attrib['v']] +=1           
    return tag

### Analyzing the values of city in tags:
Below we can see the values for city and how many times each of them occur in the OSM file

In [9]:
tag_audit(SAMPLE_FILE, "addr:city")

defaultdict(int,
            {'Alvin': 1,
             'Bay City': 1,
             'Baytown': 1,
             'Bellaire': 3,
             'Cypress': 2,
             'Deer Park': 1,
             'Galveston': 55,
             'Houston': 172,
             'Houston, Texas': 2,
             'Humble': 6,
             'Katy': 30,
             'Kingwood': 5,
             'Klein': 4,
             'League City': 1,
             'Liberty': 1,
             'Missouri City': 1,
             'Pasadena': 3,
             'Pearland': 2,
             'Seabrook': 2,
             'Stafford': 2,
             'Sugar Land': 4,
             'Texas City': 1,
             'The Woodlands': 2,
             'Tomball': 13,
             'Tomball, Tx': 5,
             'Webster': 2})

In the results above we can observe the following:
* There are a few entries that have 'Tx'/'Texas' following the city name. This has to be corrected (we will only retain the city name)
* There is one entry 'Galveston Island' which has to be changes to 'Galveston' to maintain consistency.
* The first entry '77386' seems to be a pincode value which is erroneously present here. This needs to be removed.
* 'TEXAS CITY' should be changed to 'Texas City' to maintain consistency.
* 'West University' and 'West University Place' refer to the same area. So, 'West University' should be changed to 'West University Place' to maintain consistency.
* 'Sugarland' and 'Sugar Land, TX' should be changed to 'Sugar Land' to maintain consistency as they refer to the same place.
* 'clear lake shores' should be changed to 'Clear Lake Shores' to maintain consistency as they refer to the same place.

### Analyzing the values of street names in tags:
Below we can see the values for street names and how many times each of them occur in the OSM file

In [10]:
tag_audit(SAMPLE_FILE, 'addr:street')

defaultdict(int,
            {'74th Street': 1,
             '80th Street': 2,
             'Airline Drive': 2,
             'Allenbrook Drive': 1,
             'Avenue F': 1,
             'Avenue N 1/2': 1,
             'Avenue P 1/2': 1,
             'Avenue R': 1,
             'Barraud Court': 2,
             'Bay Street North': 1,
             'Bayport Boulevard': 1,
             'Beaudelaire Circle': 1,
             'Beechnut Street': 1,
             'Beltway 8 & Wilson Road': 1,
             'Beluche Drive': 9,
             'Binz Street': 2,
             'Bissonet Street': 1,
             'Blossom': 1,
             'Briar Forest Drive': 1,
             'Bucktrout Lane': 5,
             'Candlewood Drive': 1,
             'Capitol Street': 1,
             'Carters Grove Lane': 1,
             'Carvel Lane': 1,
             'Castlewood Street': 1,
             'Cavalcade Street': 1,
             'Cinco Ranch Boulevard': 1,
             'Clara Barton Lane': 3,
             'Clay Roa

In the results above, we can observe the following:
* Abbreviations in street names like Dr, Blvd, Pkwy, Fwy need to be corrected (Eg: Dr -> Drive, Blvd -> Boulevard)
* Abbreviations like E,W,N,S need to be cahnged to East, West, North, South respectively
* All upper and lower case names need to be changed to camel case to maintain consistency
* Names with Farm-to-Market Road needs to be changed to "FM" to maintain consistency

### Analyzing the country values in the tags:
Below we can see the values for country and how many times each of them occur in the OSM file

In [11]:
tag_audit(OSM_FILE, 'addr:country')

defaultdict(int, {'US': 289})

As we can see above, all the country names in the tags are correct and consistent. Thus, there is no need to audit this field further.

### Analyzing the house numbers in the tags:
Below we can see the values for house numbers and how many times each of them occur in the OSM file

In [12]:
tag_audit(SAMPLE_FILE, 'addr:housenumber')

defaultdict(int,
            {'100': 1,
             '1000': 1,
             '10103': 1,
             '1011': 1,
             '1020': 1,
             '10700': 1,
             '10843': 1,
             '110': 1,
             '11211': 1,
             '11511 1/2': 1,
             '11815': 1,
             '11949': 1,
             '1200': 1,
             '12211': 1,
             '12424': 1,
             '12500': 1,
             '12555': 1,
             '12754': 1,
             '12901': 1,
             '1300': 1,
             '13102': 1,
             '1313': 2,
             '13210': 1,
             '1333': 1,
             '13403': 1,
             '1360': 1,
             '13600': 1,
             '13618': 1,
             '13700': 1,
             '13768': 1,
             '14': 1,
             '1400': 1,
             '1410': 1,
             '1500': 1,
             '1506': 1,
             '1510': 1,
             '1517': 1,
             '1520': 1,
             '15210': 1,
             '1522': 1,
  

From the above results, we can observe that some of the house numbers have street names in them. These need to be corrected.
For eg: "600 jefferson st" -> "600"

### Analyzing the pincodes in the tags
Below we can see the values for pincodes and how many times each of them occur in the OSM file

In [13]:
tag_audit(OSM_FILE, 'addr:postcode')

defaultdict(int,
            {'73032': 1,
             '74404': 1,
             '77002': 45,
             '77003': 5,
             '77004': 25,
             '77005': 12,
             '77005-1890': 1,
             '77006': 34,
             '77007': 19,
             '77007-2112': 1,
             '77007-2113': 1,
             '77007-2121': 1,
             '77008': 11,
             '77009': 8,
             '77010': 11,
             '77011': 4,
             '77012': 1,
             '77014': 8,
             '77015': 2,
             '77016': 1,
             '77018': 5,
             '77019': 5,
             '77020': 7,
             '77021': 4,
             '77022': 17,
             '77024': 40,
             '77024-8022': 1,
             '77025': 9,
             '77025-9998': 1,
             '77027': 16,
             '77028': 1,
             '77030': 45,
             '77031': 2,
             '77032': 13,
             '77034': 2,
             '77035': 5,
             '77036': 10,
             '7

### Analyzing the state names in the tags:
Below we can see the values for states and how many times each of them occur in the OSM file

In [20]:
tag_audit(OSM_FILE, 'addr:state')

defaultdict(int,
            {'TEXAS': 1,
             'TX': 4051,
             'TX - Texas': 1,
             'Texas': 73,
             'Tx': 79,
             'Tx.': 7,
             'texas': 3,
             'tx': 5})

As we can see from the results above, all of the values have to be changed to 'TX' (which is the most common) to maintain consistency 

In [26]:
def is_street_name(elem):
    return ((elem.tag == "tag") and (elem.attrib['k'] == "addr:street"))

def is_state(elem):
    return ((elem.tag == "tag") and (elem.attrib['k'] == "addr:state" or elem.attrib['k'] == "is_in:state_code"))

def is_postcode(elem):
    return ((elem.tag == "tag") and (elem.attrib['k'] == "addr:postcode"))

def is_housenumber(elem):
    return ((elem.tag == "tag") and (elem.attrib['k'] == "addr:housenumber"))

def is_city(elem):
    return ((elem.tag == "tag") and (elem.attrib['k'] == 'addr:city'))


In [17]:
def audit_street_type(street_types, street_name):
    street_type_re = re.compile(r'\b\S+\.?\s?$', re.IGNORECASE)
    expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road",\
                "Trail", "Parkway", "Commons", "East", "West", "North", "South", "Freeway", "Highway", "Circle", "Park"]
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [21]:
def audit_state_type(state_types, state):
    if (state != 'TX'):
        state_types.append(state)

In [22]:
def audit_postcode(incorrect_postcodes, postcode):
    postcode_re = re.compile(r'^(77...)') # Postcode outside Houston, TX
    m = postcode_re.search(postcode)
    if not m:
        incorrect_postcodes.append(postcode)
    return incorrect_postcodes

In [33]:
def audit_housenumber(house_numbers, number):
    house_number_re = re.compile(r'^\d+(-?\d)*$')
    m = house_number_re.search(number)
    if not m:
        house_numbers.append(number)
    return house_numbers

In [34]:
def audit_city(invalid_city_names, city_name):
    expected = ["Alvin", "Angleton", "Atascocita", "Bay City", "Baytown", "Bellaire", "Brazoria", "Channelview", "Clear Lake Shores", 
            "Clute", "Conroe", "Crosby", "Cypress", "Deer Park", "Dickinson", "Freeport", "Fresno'", "Friendswood", 
            "Galveston", "Hedwig Village", "Hockley", "Houston", "Humble", "Katy", "Kemah", "Kingwood", "Klein", "La Porte", "LaMarque",
            "Lake Jackson", "League City", "Liberty", "Magnolia", "Meadows Place", "Missouri City", "Nassau Bay", "Needville", 
            "Pasadena", "Pearland", "Porter", "Richmond", "Rosenberg", "Santa Fe", "Seabrook", "Shenandoah", "Spring", "Stafford", 
            "Sugar Land", "Texas City", "The Woodlands", "Tomball", "Webster", "West Columbia", "West University Place", "Winnie"]

    if city_name not in expected:
        invalid_city_names[city_name] +=1
    return invalid_city_names

In [36]:
def audit_fields(osmfile):
    
    osm_file = open(osmfile, 'r')
    incorrect_street_types = defaultdict(set) 
    # This is a dictionary of street types which contain the street values 
    # that are not in the expected list
    
    incorrect_state_types = []
    # This is a list that contains values of states other than 'TX'
    
    incorrect_postcodes = []
    # This list contains values of postcodes that are not in Houston or are invalid
    
    incorrect_House_Numbers = []
    # This list contains values of house numbers that are invalid
    
    incorrect_city_names = defaultdict(int)
    # This dictionary contains all the invalid city names
    
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(incorrect_street_types, elem.attrib['v'])
        elif is_state(elem):
            audit_state_type(incorrect_state_types, elem.attrib['v'])
        elif is_postcode(elem):
            audit_postcode(incorrect_postcodes, elem.attrib['v'])
        elif is_housenumber(elem):
            audit_housenumber(incorrect_House_Numbers, elem.attrib['v'])
        elif is_city(elem):
            audit_city(incorrect_city_names, elem.attrib['v'])
            
    print "Incorrect Street Types: "        
    pprint.pprint(incorrect_street_types)
    print "Incorrect State: "
    pprint.pprint(incorrect_state_types)
    print "Incorrect PostCode: "
    pprint.pprint(incorrect_postcodes)
    print "Incorrect House Numbers: "
    pprint.pprint(incorrect_House_Numbers)
    print "Incorrect City Names: "
    pprint.pprint(incorrect_city_names)

audit_fields(SAMPLE_FILE)

Incorrect Street Types: 
defaultdict(<type 'set'>, {'1142': set(['Lake Woodlands Drive #1142']), 'Blossom': set(['Blossom', 'Magnolia Blossom']), '2920': set(['FM 2920']), 'St.': set(['Dallas St.']), 'Maroneal': set(['Maroneal']), 'DRIVE': set(['STONEY CLOUD DRIVE']), 'Rd': set(['Rogerdale Rd']), '110': set(['Memorial Drive, Ste 110']), 'Way': set(['Frassati Way']), '1488': set(['FM 1488']), 'Hillcroft': set(['Hillcroft']), '1960': set(['FM 1960']), '6': set(['S Highway 6']), '1462': set(['Farm-to-Market Road 1462']), 'Plaza': set(['One Baylor Plaza']), '1/2': set(['Avenue P 1/2', 'Avenue N 1/2']), 'G': set(['W 34th St #G']), 'F': set(['Avenue F', 'San Felipe Road, suite F']), 'Tunnel': set(['Lamar Tunnel']), '185': set(['I-45 South, Suite 185']), 'N': set(['East Sam Houston Pkwy N']), 'R': set(['Avenue R']), 'T': set(['Avenue T']), 'Crossing': set(['Woodland Field Crossing']), '90': set(['US 90']), 'St': set(['Milby St']), 'Welford': set(['Welford']), 'Ave': set(['N Houston Ave']), '9